In [20]:
import struct
import json
import numpy as np

# File path (update as needed)
file_dir  = "./gl05rl01pe08_org/"
file_base = "boundary_GL05RL01.pe0000"
file_num  = "07"
file_path = file_dir + file_base + file_num

# Mapping from datatype code to NumPy dtype
dtype_map = {
    0: "f4",  # real4 -> float32
    1: "f8",  # real8 -> float64
    2: "i4",  # int4 -> int32
    3: "i8"   # int8 -> int64
}

# Initialize dictionary to store data
data_arrays = {}

with open(file_path, "rb") as f:
    f.read(64)  # Skip header (64 bytes)
    f.read(256)  # Skip note (256 bytes)

    # Read and discard 6 integers before rgnid
    _, _, _, _, _, num_of_rgn = struct.unpack(">6I", f.read(4 * 6))

    # Skip rgnid array (num_of_rgn * 4 bytes)
    f.read(4 * num_of_rgn)
    
    # Read num_of_data
    num_of_data = struct.unpack(">I", f.read(4))[0]

    # Process 9 datasets
    for _ in range(9):
        varname = f.read(16).decode(errors="ignore").strip().replace("\x00", "")  # Remove null characters
        f.read(64 + 16 + 16 + 256)  # Skip metadata fields

        # Read data size and type
        datasize, datatype, _, _ = struct.unpack(">Q3I", f.read(8 + 4 * 3))
        f.read(8 * 2)  # Skip time_start, time_end

        # Validate datatype
        data_format = dtype_map.get(datatype, None)
        if not data_format:
            f.seek(datasize, 1)  # Skip if unknown
            continue

        # Compute element count and read data
        element_size = np.dtype(data_format).itemsize
        num_elements = datasize // element_size
        raw_data = np.frombuffer(f.read(datasize), dtype=np.dtype(">" + data_format))

        # Ensure correct shape and convert Fortran (column-major) order to C (row-major) if needed
        reshaped_data = raw_data.reshape((5, 324), order='F').T if num_elements == 1620 else raw_data

        # Store dataset in corrected order
        data_arrays[varname] = reshaped_data.tolist()

# Save the dataset as a JSON file (correcting order issues)
output_file_path = file_base + "00" + file_num + ".json"
with open(output_file_path, "w") as json_file:
    json.dump(data_arrays, json_file, indent=4)

print(f"Corrected dataset saved to {output_file_path}")


Corrected dataset saved to boundary_GL05RL01.pe00000007.json
